
# Face Following Car

This notebook outlines how to create a face following system for a remote car. By employing face detection algorithms, the car can adjust its movements to follow a person's face within the camera's field of view.
    

In [ ]:

import cv2
import numpy as np
import socket
import json
import time
import requests
    


## Loading the Face Detection Model

We will use OpenCV's built-in Haar Cascade Classifier for face detection. This model is lightweight and suitable for real-time applications.
    

In [ ]:

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    


## Car Control Interface Configuration

Configure the car's IP address and port for sending control commands.
    

In [ ]:

IP = "192.168.4.1"
PORT = 100
    


## Function for Sending Commands to the Car

This function sends commands to the car via a socket connection, allowing for real-time control based on face position.
    

In [ ]:

def send_command(command):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((IP, PORT))
        sock.sendall(json.dumps(command).encode())
        time.sleep(0.5)
    


## Processing the Camera Stream for Face Detection

Extract frames from the camera stream, detect faces, and control the car to follow the detected face.
    

In [ ]:

stream_url = "http://192.168.4.1:81/stream"
session = requests.Session()
response = session.get(stream_url, stream=True)

if response.status_code == 200:
    bytes_data = bytes()
    while True:
        chunk = response.raw.read(1024)
        if not chunk:
            break
        bytes_data += chunk
        a = bytes_data.find(b'ÿØ')
        b = bytes_data.find(b'ÿÙ')
        while a != -1 and b != -1:
            jpg = bytes_data[a:b+2]
            bytes_data = bytes_data[b+2:]
            img = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
            
            if img is not None:
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, 1.1, 4)
                for (x, y, w, h) in faces:
                    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
                    # Face follow logic based on face position

                cv2.imshow('Face Following Car Camera Stream', img)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            a = bytes_data.find(b'ÿØ')
            b = bytes_data.find(b'ÿÙ')

cv2.destroyAllWindows()
    


## Student Exercises

1. **Implement Face Follow Logic**: Develop the logic to move the car based on the position of the detected face in the camera's field of view. Consider moving forward, backward, and turning to keep the face centered.

2. **Optimize Face Detection**: Experiment with different parameters for the face detection function to improve accuracy and reduce false positives.

3. **Integrate Additional Features**: Explore adding more features such as stopping the car if the face is too close, or executing pre-defined maneuvers when the face disappears from view.

4. **Enhance User Interaction**: Create a user-friendly interface or feedback system that indicates the car's current state and actions to the user.
    